# Introduction
This notebook estimates a VAR model for Western Australia by selected industries

# Prepare the data

In [218]:
path <- getwd()
data.WA <- read.csv(paste0(path,'/', 'ABS_5220006_WA.csv'), stringsAsFactor = FALSE)
meta.WA <- read.csv(paste0(path,'/', 'ABS_5220006_WA_META.csv'), stringsAsFactor = FALSE)

In [219]:
data.WA$Date <- seq(as.Date("1990/6/1"), as.Date("2019/6/1"), by = "year")

In [220]:
head(data.WA)

Date,A2335882X,A2335883A,A2335884C,A2335885F,A2335886J,A2335887K,A2335888L,A2335889R,A2335890X,...,A2714831X,A2714832A,A2714833C,A2714834F,A2714835J,A2714836K,A2529272W,A2528438T,A2528439V,A2528440C
<date>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1990-06-01,3517,10866,14219,4601,3794,1036,9038,979,1699,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1991-06-01,3608,10956,14418,4719,3613,954,9414,1039,1651,...,0,0,0,0,0,0,0,0,0,0
1992-06-01,3793,11050,14741,4990,3621,994,9761,1060,1757,...,0,0,0,0,0,0,0,0,0,0
1993-06-01,3911,11371,15179,5140,3826,1072,10087,1134,1956,...,0,0,0,0,0,0,0,0,0,0
1994-06-01,4083,11513,15528,5278,3462,1106,10528,1149,2120,...,0,0,0,0,0,0,0,0,0,0
1995-06-01,4236,11573,15778,5524,3494,1175,10990,1233,2067,...,0,0,0,0,0,0,0,0,0,0


In [221]:
# import and display a description of the selected variables
meta.WA.2 <- read.csv(paste0(path, '/', 'WA_Selected_Variables_VAR.csv'), stringsAsFactor = FALSE)
meta.WA.2

Description,Levels,Percent.change
<chr>,<chr>,<chr>
Financial and insurance services (K) ; Industry gross value added,A2715104W,A2714999C
Arts and recreation services (R) ; Industry gross value added,A2715111V,A2715006T
Retail trade (G) ; Industry gross value added,A2715100L,A2714995V
Accommodation and food services (H) ; Industry gross value added,A2715101R,A2714996W
"Transport, postal and warehousing (I) ; Industry gross value added",A2715102T,A2714997X
Public administration and safety (O) ; Industry gross value added,A2715108F,A2715003K
Construction (E) ; Industry gross value added,A2715098L,A2714993R
Wholesale trade (F) ; Industry gross value added,A2715099R,A2714994T
Retail trade (G) ; Industry gross value added,A2715100L,A2714995V


In [222]:
# create a dataframe with a selection of chain volume Gross Valued Added in level form (Financial & Insurance 'A2715104W', Arts & Recreation "A2715111V")
WA.Industries.Level <- names(data.WA) %in% c("A2715104W", "A2715111V") # 
WA.Industries.Level <- data.WA[ , WA.Industries.Level]
head(WA.Industries.Level)

A2715104W,A2715111V
<int>,<int>
3821,547
3800,546
4007,567
4352,598
4996,655
4886,684


In [223]:
# create a dataframe with a selection of chain volume Gross Valued Added in level form (Transport 'A2715102T, Accommodation and Food Services 'A2715101R')
WA.Industries.Trans_Accom.Level <- names(data.WA) %in% c("A2715101R", "A2715102T") # , 
WA.Industries.Trans_Accom.Level <- data.WA[ , WA.Industries.Trans_Accom.Level]
head(WA.Industries.Trans_Accom.Level)

A2715101R,A2715102T
<int>,<int>
1250,3111
1337,3054
1426,3244
1488,3233
1746,3629
1922,3756


In [224]:
# create a dataframe with a selection of chain volume Gross Valued Added in level form (construction 'A2715098L' versus transport 'A2715102T' versus wholesale trade 'A2715099R')
WA.trns_cnstrctn_whlsl <- names(data.WA) %in% c("A2715098L", "A2715102T", "A2715099R") # , 
var.WA.trns_cnstrctn_whlsl <- data.WA[ , WA.trns_cnstrctn_whlsl] #
head(var.WA.trns_cnstrctn_whlsl)

A2715098L,A2715099R,A2715102T
<int>,<int>,<int>
5735,2027,3111
4979,1876,3054
4923,1927,3244
5398,2096,3233
6421,2373,3629
5964,2570,3756


In [225]:
# create a dataframe with a selection of chain volume Gross Valued Added in percent change form: Arts and recreation services - 'A2715006T'; Retail trade - 'A2714995V'
WA.industries <- names(data.WA) %in% c("A2715006T", "A2714995V") 
var.data.WA <- data.WA[2:nrow(data.WA), WA.industries] #
head(var.data.WA)

,A2714995V,A2715006T
,<dbl>,<dbl>
2,-1.5,-0.1
3,8.6,3.8
4,7.6,5.5
5,7.0,9.5
6,3.4,4.4
7,7.7,-3.7


In [226]:
# create a selection of chain volume Gross Valued Added in levels form: Arts and recreation services - 'A2715111V'; Retail trade - 'A2715100L'
WA.Arts_Retail.Level.names <- names(data.WA) %in% c("A2715111V", "A2715100L") 
WA.Arts_Retail.Level <- data.WA[2:nrow(data.WA), WA.Arts_Retail.Level.names] #
head(WA.Arts_Retail.Level)

,A2715100L,A2715111V
,<int>,<int>
2,2578,546
3,2800,567
4,3012,598
5,3223,655
6,3333,684
7,3590,658


### Apply additional transformations to the data

In [228]:
# convert levels to logarithmic form:
WA.Industries.Trans_Accom.Level <- log(WA.Industries.Trans_Accom.Level)
var.WA.trns_cnstrctn_whlsl <- log(var.WA.trns_cnstrctn_whlsl)
var.WA.Arts_Retail.Level <- log(WA.Arts_Retail.Level)

In [229]:
# Convert to time series format

WA.Industries.Trans_Accom.Level <- ts(WA.Industries.Trans_Accom.Level)
var.WA.trns_cnstrctn_whlsl <- ts(var.WA.trns_cnstrctn_whlsl)
var.data.WA <- ts(var.data.WA) 
var.WA.Arts_Retail.Level <- ts(var.WA.Arts_Retail.Level)

In [230]:
head(WA.Industries.Trans_Accom.Level)

A2715101R,A2715102T
1.964437,2.084765
1.973829,2.082463
1.982742,2.089956
1.988585,2.089536
2.010237,2.103733
2.023020,2.107921


In [233]:
head(var.WA.Arts_Retail.Level)

A2715100L,A2715111V
7.854769,6.302619
7.937375,6.340359
8.010360,6.393591
8.078068,6.484635
8.111628,6.527958
8.185907,6.489205


# Estimate VAR models

In [179]:
install.packages("vars")
library(vars) # Load package

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [201]:
#help(VAR)

### Transport-Warehousing industry & Accommodation-Food

In [202]:
# estimate VAR in logarithmic form: Transport-Warehousing (A2715102T) industry & Accommodation-Food (A2715101R)
var.Trans_Accom.Level <- VAR(WA.Industries.Trans_Accom.Level, 1, type = "const") # Estimate the model

In [203]:
summary(var.Trans_Accom.Level)


VAR Estimation Results:
Endogenous variables: A2715101R, A2715102T 
Deterministic variables: const 
Sample size: 29 
Log Likelihood: -385.602 
Roots of the characteristic polynomial:
0.954 0.954
Call:
VAR(y = WA.Industries.Trans_Accom.Level, p = 1, type = "const")


Estimation results for equation A2715101R: 
A2715101R = A2715101R.l1 + A2715102T.l1 + const 

              Estimate Std. Error t value Pr(>|t|)    
A2715101R.l1   1.03556    0.11430   9.060 1.59e-09 ***
A2715102T.l1  -0.03207    0.05182  -0.619   0.5413    
const        197.17382   99.05648   1.991   0.0571 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 184.6 on 26 degrees of freedom
Multiple R-Squared: 0.9741,	Adjusted R-squared: 0.9721 
F-statistic: 489.4 on 2 and 26 DF,  p-value: < 2.2e-16 


Estimation results for equation A2715102T: 
A2715102T = A2715101R.l1 + A2715102T.l1 + const 

              Estimate Std. Error t value Pr(>|t|)    
A2715101R.l1   0.34374    0.16

### Summary of results
The results shown in the cell above indicates that the growth of Gross Value Added in the Transport & Warehousing (A2715102T) industry is partly explained by variation in the Accommodation & Food (A2715101R) industry 

## Construction-Transport-Wholesale Trade
This section presents results of a VAR model of variation in Gross Value Added in the Construction, Transport & Warehousing, and the Wholesale Trade industries. 

In [209]:
# estimate VAR in logarithmic form: var.WA.trns_cnstrctn_whlsl
var.WA.trns_cnstrctn_whlsl <- VAR(var.WA.trns_cnstrctn_whlsl, type = "const", lag.max = 5, ic = "AIC") # Estimate the model
summary(var.WA.trns_cnstrctn_whlsl)


VAR Estimation Results:
Endogenous variables: A2715098L, A2715099R, A2715102T 
Deterministic variables: const 
Sample size: 25 
Log Likelihood: -490.182 
Roots of the characteristic polynomial:
1.121 1.121 1.036 1.013 1.013 1.008 1.008 0.951 0.951 0.9295 0.9295 0.8944 0.8944 0.704 0.1633
Call:
VAR(y = var.WA.trns_cnstrctn_whlsl, type = "const", lag.max = 5, 
    ic = "AIC")


Estimation results for equation A2715098L: 
A2715098L = A2715098L.l1 + A2715099R.l1 + A2715102T.l1 + A2715098L.l2 + A2715099R.l2 + A2715102T.l2 + A2715098L.l3 + A2715099R.l3 + A2715102T.l3 + A2715098L.l4 + A2715099R.l4 + A2715102T.l4 + A2715098L.l5 + A2715099R.l5 + A2715102T.l5 + const 

               Estimate Std. Error t value Pr(>|t|)   
A2715098L.l1    0.56949    0.35563   1.601  0.14376   
A2715099R.l1    1.01657    1.98644   0.512  0.62114   
A2715102T.l1    2.37405    4.24309   0.560  0.58946   
A2715098L.l2   -0.40356    0.44845  -0.900  0.39161   
A2715099R.l2    2.10875    2.90502   0.726  0.48635   
A

### Summary of result
The results shown in the cell directly above indicates that variation in Gross Value Added (GVA) in the:
1. Construction industry is partly explained by variation in the Wholesale Trade industry GVA. 
2. Wholesale Trade industry is partly explained by variation in the GVA of the Construction and Transport industries.
3. Transport & Warehousing industry is partly explained by variation in the GVA of the Construction and Wholesale Trade industries.

Key: Construction 'A2715098L'; Transport & Warehousing 'A2715102T'; Wholesale Trade 'A2715099R'

In [154]:
# Calculate the IRF for the levels model: var.WA.trns_cnstrctn_whlsl (construction 'A2715098L' versus transport 'A2715102T' versus wholesale trade 'A2715099R')
ir.1 <- irf(var.WA.trns_cnstrctn_whlsl, impulse = "A2715098L", response = "A2715099R", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

### Summary of result
The result shown in the cell directly above suggests that a shock from the Construction (A2715098L) industry might have weak a negative effect on the Wholesale Trade (A2715099R') industry.

In [153]:
# Calculate the IRF for the levels model: var.WA.trns_cnstrctn_whlsl (construction 'A2715098L' versus transport 'A2715102T' versus wholesale trade 'A2715099R')
ir.1 <- irf(var.WA.trns_cnstrctn_whlsl, impulse = "A2715098L", response = "A2715102T", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

### Summary of result
The result shown directly above indicates that a shock from the Construction (A2715098L) industry has no effect on the Transport and Warehousing (A2715102T) industry.

In [156]:
# Calculate the IRF for the levels model: var.WA.trns_cnstrctn_whlsl (construction 'A2715098L' versus transport 'A2715102T' versus wholesale trade 'A2715099R')
ir.1 <- irf(var.WA.trns_cnstrctn_whlsl, impulse = "A2715102T", response = "A2715098L", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

### Summary of result
The result above indicates that a positive shock from the Transport and Warehousing (A2715102T) industry imposes a negative shock on the Construction (A2715098L) industry.  

In [211]:
# Calculate impulse response
ir.2 <- irf(var.WA.trns_cnstrctn_whlsl,impulse="A2715102T",response="A2715098L",n.ahead = 5,ortho = FALSE,
cumulative = TRUE)

# Plot
plot(ir.2)

### Summary of result
Inconclusive: The chart shown in the cell above indicates that the cumulative effect of a shock in Transport and Warehousing (A2715102T) on variation in cumulative GVA in the Construction (A2715098L) industry is not statistically significant.

In [158]:
# Calculate the IRF for the levels model: var.WA.trns_cnstrctn_whlsl (construction 'A2715098L' versus transport 'A2715102T' versus wholesale trade 'A2715099R')
ir.1 <- irf(var.WA.trns_cnstrctn_whlsl, impulse = "A2715099R", response = "A2715098L", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

### Summary of result
The result shown above indicates that a shock from the Wholesale Trade (A2715099R) industry causes an increasing shock on the Construction (A2715098L) industry. 

In [212]:
# Calculate impulse response
ir.2 <- irf(var.WA.trns_cnstrctn_whlsl,impulse="A2715099R",response="A2715098L",n.ahead = 5,ortho = FALSE,
cumulative = TRUE)

# Plot
plot(ir.2)

### Summary of result
The result shown above indicates that the cumulative effect of a shock from the Wholesale Trade (A2715099R) industry causes an increasing shock on the Construction (A2715098L) industry over a period of three to four years. 

## Arts & Recreation-Finance industry
This section presents results of VAR modelling of Gross Value Added (GVA) between the Arts & Recreation and Finance industry.

In [205]:
var.aic <- VAR(var.data.WA, type = "none", lag.max = 5, ic = "AIC")
summary(var.aic)


VAR Estimation Results:
Endogenous variables: A2714995V, A2715006T 
Deterministic variables: none 
Sample size: 28 
Log Likelihood: -159.469 
Roots of the characteristic polynomial:
0.7995 0.04352
Call:
VAR(y = var.data.WA, type = "none", lag.max = 5, ic = "AIC")


Estimation results for equation A2714995V: 
A2714995V = A2714995V.l1 + A2715006T.l1 

             Estimate Std. Error t value Pr(>|t|)    
A2714995V.l1   0.6879     0.1434   4.798 5.73e-05 ***
A2715006T.l1   0.1226     0.1127   1.088    0.286    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 3.35 on 26 degrees of freedom
Multiple R-Squared: 0.6121,	Adjusted R-squared: 0.5823 
F-statistic: 20.51 on 2 and 26 DF,  p-value: 4.5e-06 


Estimation results for equation A2715006T: 
A2715006T = A2714995V.l1 + A2715006T.l1 

             Estimate Std. Error t value Pr(>|t|)  
A2714995V.l1  0.66613    0.24315   2.740    0.011 *
A2715006T.l1  0.06816    0.19112   0.357    0.724  
---
Sig

### Summary of results
The results shown in the cell above indicates that variation in the growth of Gross Value Added (GVA) for Retail trade (A2714995V) partly explains the variation in the growth of Arts and Recreation Services (A2715006T) industry GVA.  

In [238]:
# estimate VAR in log-levels form
# Arts and recreation services - 'A2715111V'; Retail trade - 'A2715100L'
var.1 <- VAR(var.WA.Arts_Retail.Level, type = "const", lag.max=5, ic='AIC' ) # Estimate the model
summary(var.1)


VAR Estimation Results:
Endogenous variables: A2715100L, A2715111V 
Deterministic variables: const 
Sample size: 28 
Log Likelihood: 113.349 
Roots of the characteristic polynomial:
0.8933 0.8933
Call:
VAR(y = var.WA.Arts_Retail.Level, type = "const", lag.max = 5, 
    ic = "AIC")


Estimation results for equation A2715100L: 
A2715100L = A2715100L.l1 + A2715111V.l1 + const 

             Estimate Std. Error t value Pr(>|t|)    
A2715100L.l1  1.02636    0.03548  28.929  < 2e-16 ***
A2715111V.l1 -0.09834    0.04354  -2.259   0.0329 *  
const         0.48068    0.09826   4.892 4.93e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.02184 on 25 degrees of freedom
Multiple R-Squared: 0.9964,	Adjusted R-squared: 0.9961 
F-statistic:  3450 on 2 and 25 DF,  p-value: < 2.2e-16 


Estimation results for equation A2715111V: 
A2715111V = A2715100L.l1 + A2715111V.l1 + const 

             Estimate Std. Error t value Pr(>|t|)    
A2715100L.l1  0.

### Summary of result
The results shown in the cell directly above indicates that there is two-way causation of variation in Gross Value Added between the Arts and Recreation Services industry and the Retail Trade industry.

Key: Arts and recreation services - 'A2715111V'; Retail trade - 'A2715100L'

In [239]:
# Calculate the IRF for the levels model
ir.1 <- irf(var.1, impulse = "A2715111V", response = "A2715100L", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

### Summary of result
Inconclusive: The impulse response chart shown in the cell above indicates that a shock in Gross Value Added (GVA) in Retail trade (A2715100L) probably does not affect variation in the Arts and Recreation Services (A2715100L) industry.

Key: Arts and recreation services - 'A2715111V'; Retail trade - 'A2715100L'

In [240]:
# Calculate the IRF for the levels model
ir.1 <- irf(var.1, impulse = "A2715100L", response = "A2715111V", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

### Summary of result
Inconclusive: The impulse response chart shown in the cell above indicates that a shock in Gross Value Added (GVA) in the Arts and Recreation Services (A2715111V) industry probably does not affect variation in the Retail trade (A2715100L).

Key: Arts and recreation services - 'A2715111V'; Retail trade - 'A2715100L'

In [76]:
# Calculate the IRF for the levels model: Accommodation-Food industry to Transport-Warehousing industry
ir.1 <- irf(var.Trans_Accom.Level, impulse = "A2715101R", response = "A2715102T", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

In [77]:
# Calculate the IRF for the levels model: Transport-Warehousing industry to Accommodation-Food industry 
ir.1 <- irf(var.Trans_Accom.Level, impulse = "A2715102T", response = "A2715101R", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

In [44]:
# Calculate the IRF for the levels model
ir.1 <- irf(var.Level.aic, impulse = "A2715111V", response = "A2715104W", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

In [155]:
# Calculate the IRF for the levels model: var.WA.trns_cnstrctn_whlsl (construction 'A2715098L' versus transport 'A2715102T' versus wholesale trade 'A2715099R')
ir.1 <- irf(var.WA.trns_cnstrctn_whlsl, impulse = "A2715102T", response = "A2715099R", n.ahead = 5, ortho = FALSE) # 

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

In [42]:
# Calculate the IRF
ir.1 <- irf(var.aic, impulse = "A2715006T", response = "A2715006T", n.ahead = 5, ortho = FALSE) # "A2714995V"

# Plot the IRF
library(repr)
options(repr.plot.width=6, repr.plot.height=5)

plot(ir.1)

### Summary of result
Shock from Arts and Recreation has no long-run effect on itself